# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [6]:
# Write your code below.
%load_ext dotenv
%dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [7]:
import dask.dataframe as dd

/tmp/ipykernel_131332/676544213.py:1: DeprecationWarning: The current Dask DataFrame implementation is deprecated. 
In a future release, Dask DataFrame will use a new implementation that
contains several improvements including a logical query planning.
The user-facing DataFrame API will remain unchanged.

The new implementation is already available and can be enabled by
installing the dask-expr library:

    $ pip install dask-expr

and turning the query planning option on:

    >>> import dask
    >>> dask.config.set({'dataframe.query-planning': True})
    >>> import dask.dataframe as dd

API documentation for the new implementation is available at
https://docs.dask.org/en/stable/dask-expr-api.html

Any feedback can be reported on the Dask issue tracker
https://github.com/dask/dask/issues 

To disable this warning in the future, set dask config:

    # via Python
    >>> dask.config.set({'dataframe.query-planning-warning': False})

    # via CLI
    dask config set dataframe.query-pla

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [13]:
import os
from glob import glob

# Write your code below.
stock_files = glob(os.path.join(os.getenv('PRICE_DATA'), "*/*/*.parquet"))
stock_files

['../../05_src/data/prices/TNC/TNC_1973/part.0.parquet',
 '../../05_src/data/prices/TNC/TNC_1973/part.1.parquet',
 '../../05_src/data/prices/TNC/TNC_1974/part.0.parquet',
 '../../05_src/data/prices/TNC/TNC_1974/part.1.parquet',
 '../../05_src/data/prices/TNC/TNC_1975/part.0.parquet',
 '../../05_src/data/prices/TNC/TNC_1975/part.1.parquet',
 '../../05_src/data/prices/TNC/TNC_1976/part.1.parquet',
 '../../05_src/data/prices/TNC/TNC_1976/part.0.parquet',
 '../../05_src/data/prices/TNC/TNC_1977/part.1.parquet',
 '../../05_src/data/prices/TNC/TNC_1977/part.0.parquet',
 '../../05_src/data/prices/TNC/TNC_1978/part.1.parquet',
 '../../05_src/data/prices/TNC/TNC_1978/part.0.parquet',
 '../../05_src/data/prices/TNC/TNC_1979/part.1.parquet',
 '../../05_src/data/prices/TNC/TNC_1979/part.0.parquet',
 '../../05_src/data/prices/TNC/TNC_1980/part.0.parquet',
 '../../05_src/data/prices/TNC/TNC_1980/part.1.parquet',
 '../../05_src/data/prices/TNC/TNC_1981/part.0.parquet',
 '../../05_src/data/prices/TNC/

For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [ ]:
# Write your code below.
ddf = dd.read_parquet(stock_files)
ddf2 = ddf.groupby('ticker', group_keys=False).apply(
    lambda x: x.assign(
        Close_lag_1 = x['Close'].shift(1),
        Adj_Close_lag_1 = x['Adj Close'].shift(1)
    ).assign(
        returns = lambda x: x['Close'] / x['Close_lag_1'] - 1,
        hi_lo_range = lambda x: x['High'] - x['Low']
    )
)
# ddf2.compute()

/tmp/ipykernel_131332/170270118.py:3: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  ddf2 = ddf.groupby('ticker', group_keys=False).apply(


+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [55]:
# Write your code below.
pdf = ddf2.compute().reset_index(drop=True)
pdf['returns_moving_10'] = pdf.groupby('ticker')['returns'].rolling(10).mean().reset_index(level=0, drop=True)
pdf

,Date,Open,High,Low,Close,Adj Close,Volume,source,ticker,Year,Close_lag_1,Adj_Close_lag_1,returns,hi_lo_range,returns_moving_10
0,2020-02-19,24.655001,24.705999,24.655001,24.680000,24.680000,3100.0,CHGX.csv,CHGX,2020,NaN,NaN,NaN,0.050999,NaN
1,2020-02-20,24.518000,24.549999,24.518000,24.549999,24.549999,600.0,CHGX.csv,CHGX,2020,24.680000,24.680000,-0.005267,0.032000,NaN
2,2020-02-21,24.320000,24.360001,24.230000,24.288000,24.288000,7900.0,CHGX.csv,CHGX,2020,24.549999,24.549999,-0.010672,0.130001,NaN
3,2020-02-24,24.000000,24.000000,23.620001,23.629999,23.629999,9300.0,CHGX.csv,CHGX,2020,24.288000,24.288000,-0.027092,0.379999,NaN
4,2020-02-25,23.969999,23.969999,22.813000,22.813000,22.813000,26400.0,CHGX.csv,CHGX,2020,23.629999,23.629999,-0.034575,1.157000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315623,2017-12-22,102.750000,102.750000,101.849998,102.550003,96.965042,153900.0,SMG.csv,SMG,2017,102.430000,96.851570,0.001172,0.900002,0.001059
315624,2017-12-26,102.750000,103.190002,102.250000,102.970001,97.362160,126200.0,SMG.csv,SMG,2017,102.550003,96.965042,0.004096,0.940002,0.000306
315625,2017-12-27,103.019997,103.389999,102.809998,103.070000,97.456718,326700.0,SMG.csv,SMG,2017,102.970001,97.362160,0.000971,0.580002,0.000364
315626,2017-12-28,103.709999,105.089996,103.400002,104.959999,99.243782,449700.0,SMG.csv,SMG,2017,103.070000,97.456718,0.018337,1.689995,0.002432


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

In [ ]:
In order to answer this question, I tried to do it in dask with the following code:

```
ddf2.groupby('ticker')['returns'].rolling(10).mean().reset_index(level=0, drop=True).compute()
```

But I got the following error:

```
ValueError: Only time indexes are supported for rolling groupbys in dask dataframe. ``window`` must be a ``freq`` (e.g. '1H').
```

From what I gathered, dusk does not support rolling windows unless there's a proper time based index, so maybe the rolling method doesn't work directly, but other approaches with for loops would.

I assume it's better to do it in Dask due to the volume of data and of the computational load of transforming one data set to the other technology. Dask is faster for most use cases so it makes sense to use it as much as possible.

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.